# Genetic Algorithm

In this project, the aim is to solve a sudoku puzzle using the genetic algorithm

In [10]:
import random as rndm
import numpy as np

## Part 1: Defining Genes and Chromosomes

A gene is the value of a row and is a premutation of set {1 ... 9}. A chromosome consists of 9 genes, each gene representing a row of the actual sudoku puzzle.
Reason TBA

In [17]:
def make_gene():
    gene=list(range(1,10));
    rndm.shuffle(gene);
    return gene;

In [18]:
def make_chromosome():
    chromosome=[]
    for _ in range(9):
        chromosome.append(make_gene());
    return chromosome

In [19]:
print(make_chromosome())

[[9, 7, 6, 8, 5, 4, 1, 3, 2], [7, 6, 9, 8, 3, 1, 4, 5, 2], [8, 4, 9, 6, 1, 5, 2, 7, 3], [2, 8, 7, 4, 3, 9, 6, 5, 1], [3, 1, 4, 5, 6, 7, 9, 2, 8], [9, 1, 4, 7, 3, 6, 2, 8, 5], [2, 4, 3, 6, 9, 5, 1, 7, 8], [4, 1, 8, 2, 3, 5, 7, 6, 9], [5, 2, 9, 3, 8, 7, 4, 6, 1]]


## Part 2: Making First Generation

In this part, a function is implemented to randomly create a population.

In [22]:
POPULATION=10

In [26]:
def make_population(count):
    population=[]
    for _ in range(count):
        population.append(make_chromosome())
    return population

In [28]:
print(make_population(POPULATION))

[[[4, 6, 7, 2, 9, 5, 3, 8, 1], [5, 4, 3, 9, 2, 6, 7, 8, 1], [7, 1, 2, 6, 3, 8, 4, 9, 5], [9, 7, 8, 3, 1, 5, 4, 2, 6], [1, 5, 6, 3, 7, 9, 4, 2, 8], [3, 8, 2, 4, 6, 7, 9, 1, 5], [4, 7, 2, 8, 3, 5, 6, 9, 1], [6, 4, 8, 2, 7, 5, 3, 9, 1], [2, 9, 1, 7, 8, 6, 5, 4, 3]], [[7, 8, 9, 3, 1, 4, 5, 6, 2], [1, 7, 4, 2, 5, 9, 3, 6, 8], [4, 2, 7, 9, 5, 1, 8, 6, 3], [4, 6, 5, 8, 3, 7, 1, 9, 2], [6, 8, 7, 3, 5, 4, 2, 9, 1], [3, 6, 8, 4, 5, 7, 2, 1, 9], [2, 4, 7, 5, 1, 9, 8, 6, 3], [7, 9, 5, 3, 4, 6, 8, 2, 1], [2, 9, 6, 8, 4, 3, 7, 5, 1]], [[3, 8, 5, 1, 2, 9, 6, 4, 7], [1, 8, 3, 5, 6, 4, 7, 9, 2], [6, 4, 3, 2, 7, 5, 1, 9, 8], [4, 2, 1, 3, 7, 5, 6, 8, 9], [9, 2, 6, 7, 1, 4, 3, 5, 8], [1, 2, 4, 8, 5, 6, 3, 7, 9], [8, 2, 9, 1, 6, 7, 5, 3, 4], [6, 9, 8, 3, 7, 1, 5, 2, 4], [4, 9, 1, 5, 8, 6, 3, 2, 7]], [[8, 6, 1, 9, 5, 3, 2, 7, 4], [8, 7, 9, 3, 4, 5, 2, 6, 1], [8, 5, 3, 4, 1, 6, 2, 7, 9], [3, 1, 7, 5, 9, 8, 2, 6, 4], [5, 3, 2, 8, 4, 9, 7, 6, 1], [2, 4, 7, 6, 8, 3, 5, 9, 1], [6, 2, 1, 7, 8, 9, 4, 3, 5], [2, 9,